In [228]:
import os
from glob import glob
import pandas as pd
pd.set_option('display.max_colwidth', None)



In [284]:
reports = dict(map(lambda fpath: (fpath, pd.read_json(fpath)), glob('reports/*/*')))
df = pd.concat(reports)
df.index.set_names(['fpath','test_idx'],inplace=True)
df.fullName = df.fullName.str.replace('Evaluation - Phase 1 - Using Web Resolvable DID Document ','')
df.fullName  = df.fullName.str.replace('Evaluation - Phase 1 - Using did:key ','')
df.fullName = df.fullName.str.strip()
df.index.set_names(['fpath','test_idx'],inplace=True)
df.reset_index(inplace=True)
df['suite'] = df.fpath.str.split('/').apply(lambda x:x[1])
df_dk = df[df.suite == 'did-key']
df_wr = df[df.suite == 'web-resolvable']
df_dk = df_dk.reset_index().drop('index',axis=1)
df_dk.index.set_names('idx',inplace=True)
df_dk = df_dk.reset_index().set_index(['idx','test_idx'])
df_wr = df_wr.reset_index().drop('index',axis=1)
df_wr.index.set_names('idx',inplace=True)
df_wr = df_wr.reset_index().set_index(['idx','test_idx'])
dfx = df_dk.merge(df_wr,left_index=True, right_index=True, suffixes=('_dk','_wr'))
dfx

,,fpath_dk,duration_dk,fullName_dk,status_dk,suite_dk,fpath_wr,duration_wr,fullName_wr,status_wr,suite_wr
idx,test_idx,,,,,,,,,,
0,0,reports/did-key/performance-report_26.json,583,[recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_26.json,112,[recruiter] verifies VC created by [alice],passed,web-resolvable
1,1,reports/did-key/performance-report_26.json,478,[recruiter] verifies VP created by [alice],passed,did-key,reports/web-resolvable/performance-report_26.json,67,[recruiter] verifies VP created by [alice],passed,web-resolvable
2,2,reports/did-key/performance-report_26.json,2815,[recruiter] verifies VP with 1 derived VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_26.json,99,[recruiter] verifies VP with 1 derived VC created by [alice],passed,web-resolvable
3,3,reports/did-key/performance-report_26.json,4799,[recruiter] verifies VP with 2 derived VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_26.json,150,[recruiter] verifies VP with 2 derived VC created by [alice],passed,web-resolvable
4,4,reports/did-key/performance-report_26.json,2539,[government] issues VC01 to [alice] ; [alice] creates VP with derived VC01 ; [recruiter] verifies VP,passed,did-key,reports/web-resolvable/performance-report_26.json,88,[government] issues VC01 to [alice] ; [alice] creates VP with derived VC01 ; [recruiter] verifies VP,passed,web-resolvable
...,...,...,...,...,...,...,...,...,...,...,...
175,1,reports/did-key/performance-report_25.json,496,[recruiter] verifies VP created by [alice],passed,did-key,reports/web-resolvable/performance-report_25.json,68,[recruiter] verifies VP created by [alice],passed,web-resolvable
176,2,reports/did-key/performance-report_25.json,2830,[recruiter] verifies VP with 1 derived VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_25.json,99,[recruiter] verifies VP with 1 derived VC created by [alice],passed,web-resolvable
177,3,reports/did-key/performance-report_25.json,4720,[recruiter] verifies VP with 2 derived VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_25.json,148,[recruiter] verifies VP with 2 derived VC created by [alice],passed,web-resolvable


In [291]:
import numpy as np

In [342]:
agg_functions = ['count', 'min','max', 'mean','std',]
dfagg = dfx.filter(regex='duration*').groupby('test_idx').agg(agg_functions).round(2)

dfagg_dk = dfagg['duration_dk'].rename({k: f'{k}[ms]' for k in agg_functions if k != 'count'},axis=1)
dfagg_wr = dfagg['duration_wr'].rename({k: f'{k}[ms]' for k in agg_functions if k != 'count'},axis=1)

In [343]:
dfagg_wr

,count,min[ms],max[ms],mean[ms],std[ms]
test_idx,,,,,
0,30,83,121,104.77,9.33
1,30,63,71,66.27,2.16
2,30,95,161,101.60,11.52
3,30,140,183,149.70,9.69
4,30,85,104,90.57,4.49
5,30,141,190,156.77,10.85


In [349]:

pd.concat({'Alice uses her anonymous did:key identifier to manage & exhange her credentials': testcases.to_frame().merge(dfagg_dk,left_index=True,right_index=True)}, axis=1)

Alice uses her anonymous did:key identifier to manage & exhange her credentials                                       
                                                                                                                                                                            fullName count min[ms] max[ms] mean[ms] std[ms]
test_idx                                                                                                                                                                                                                   
0                                                                                                                                         [recruiter] verifies VC created by [alice]    30     583     955   698.90   85.32
1                                                                                                                                         [recruiter] verifies VP created by [alice]    30     478     851   551.37  100.76
2                                                                                                                       [recruiter] verifies VP with 1 derived VC created by [alice]    30    2805    3959  3020.37  220.73
3                                                                                                                       [recruiter] verifies VP with 2 derived VC created by [alice]    30    4447    4998  4730.77  122.01
4                                                                               [government] issues VC01 to [alice] ; [alice] creates VP with derived VC01 ; [recruiter] verifies VP    30    2419    4575  2658.90  378.17
5         [government] issues VC01 to [alice] ;\n    [university] issues VC02 to [alice] ; \n    [alice] creates VP with derived VC01 & derived VC02 ; \n    [recruiter] verifies VP    30    4452    5714  4716.03  228.43

In [345]:

pd.concat({'Alice uses a DID Document hosted on her Solid Pod identifier to manage & exhange her credentials': testcases.to_frame().merge(dfagg_wr,left_index=True,right_index=True)}, axis=1)

Alice uses a DID Document hosted on her Solid Pod identifier to manage & exhange her credentials                                       
                                                                                                                                                                            fullName count min[ms] max[ms] mean[ms] std[ms]
test_idx                                                                                                                                                                                                                   
0                                                                                                                                         [recruiter] verifies VC created by [alice]    30      83     121   104.77    9.33
1                                                                                                                                         [recruiter] verifies VP created by [alice]    30      63      71    66.27    2.16
2                                                                                                                       [recruiter] verifies VP with 1 derived VC created by [alice]    30      95     161   101.60   11.52
3                                                                                                                       [recruiter] verifies VP with 2 derived VC created by [alice]    30     140     183   149.70    9.69
4                                                                               [government] issues VC01 to [alice] ; [alice] creates VP with derived VC01 ; [recruiter] verifies VP    30      85     104    90.57    4.49
5         [government] issues VC01 to [alice] ;\n    [university] issues VC02 to [alice] ; \n    [alice] creates VP with derived VC01 & derived VC02 ; \n    [recruiter] verifies VP    30     141     190   156.77   10.85

In [293]:
dfx.loc(axis=0)[pd.IndexSlice[:],0]

,,fpath_dk,duration_dk,fullName_dk,status_dk,suite_dk,fpath_wr,duration_wr,fullName_wr,status_wr,suite_wr
idx,test_idx,,,,,,,,,,
0,0,reports/did-key/performance-report_26.json,583,[recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_26.json,112,[recruiter] verifies VC created by [alice],passed,web-resolvable
6,0,reports/did-key/performance-report_30.json,905,[recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_30.json,93,[recruiter] verifies VC created by [alice],passed,web-resolvable
12,0,reports/did-key/performance-report_10.json,646,[recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_10.json,102,[recruiter] verifies VC created by [alice],passed,web-resolvable
18,0,reports/did-key/performance-report_11.json,605,[recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_11.json,98,[recruiter] verifies VC created by [alice],passed,web-resolvable
24,0,reports/did-key/performance-report_1.json,955,[recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_1.json,108,[recruiter] verifies VC created by [alice],passed,web-resolvable
30,0,reports/did-key/performance-report_27.json,680,[recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_27.json,113,[recruiter] verifies VC created by [alice],passed,web-resolvable
36,0,reports/did-key/performance-report_6.json,692,[recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_6.json,99,[recruiter] verifies VC created by [alice],passed,web-resolvable
42,0,reports/did-key/performance-report_16.json,680,[recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_16.json,108,[recruiter] verifies VC created by [alice],passed,web-resolvable
48,0,reports/did-key/performance-report_20.json,666,[recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_20.json,121,[recruiter] verifies VC created by [alice],passed,web-resolvable


,,fpath_x,duration_x,fullName_x,status_x,suite_x,fpath_y,duration_y,fullName_y,status_y,suite_y
idx,test_idx,,,,,,,,,,
0,0,reports/did-key/performance-report_1.json,695,Evaluation - Phase 1 - Using did:key [recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_1.json,90,Evaluation - Phase 1 - Using Web Resolvable DID Document [recruiter] verifies VC created by [alice],passed,web-resolvable
1,1,reports/did-key/performance-report_1.json,506,Evaluation - Phase 1 - Using did:key [recruiter] verifies VP created by [alice],passed,did-key,reports/web-resolvable/performance-report_1.json,70,Evaluation - Phase 1 - Using Web Resolvable DID Document [recruiter] verifies VP created by [alice],passed,web-resolvable
2,2,reports/did-key/performance-report_1.json,3032,Evaluation - Phase 1 - Using did:key [recruiter] verifies VP with 1 derived VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_1.json,101,Evaluation - Phase 1 - Using Web Resolvable DID Document [recruiter] verifies VP with 1 derived VC created by [alice],passed,web-resolvable
3,3,reports/did-key/performance-report_1.json,4771,Evaluation - Phase 1 - Using did:key [recruiter] verifies VP with 2 derived VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_1.json,150,Evaluation - Phase 1 - Using Web Resolvable DID Document [recruiter] verifies VP with 2 derived VC created by [alice],passed,web-resolvable
4,4,reports/did-key/performance-report_1.json,2639,Evaluation - Phase 1 - Using did:key [government] issues VC01 to [alice] ; [alice] creates VP with derived VC01 ; [recruiter] verifies VP,passed,did-key,reports/web-resolvable/performance-report_1.json,89,Evaluation - Phase 1 - Using Web Resolvable DID Document [government] issues VC01 to [alice] ; [alice] creates VP with derived VC01 ; [recruiter] verifies VP,passed,web-resolvable
5,5,reports/did-key/performance-report_1.json,5078,Evaluation - Phase 1 - Using did:key [government] issues VC01 to [alice] ;\n [university] issues VC02 to [alice] ; \n [alice] creates VP with derived VC01 & derived VC02 ; \n [recruiter] verifies VP,passed,did-key,reports/web-resolvable/performance-report_1.json,166,Evaluation - Phase 1 - Using Web Resolvable DID Document [government] issues VC01 to [alice] ;\n [university] issues VC02 to [alice] ; \n [alice] creates VP with derived VC01 & derived VC02 ; \n [recruiter] verifies VP,passed,web-resolvable
6,0,reports/did-key/performance-report_2.json,704,Evaluation - Phase 1 - Using did:key [recruiter] verifies VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_2.json,89,Evaluation - Phase 1 - Using Web Resolvable DID Document [recruiter] verifies VC created by [alice],passed,web-resolvable
7,1,reports/did-key/performance-report_2.json,544,Evaluation - Phase 1 - Using did:key [recruiter] verifies VP created by [alice],passed,did-key,reports/web-resolvable/performance-report_2.json,65,Evaluation - Phase 1 - Using Web Resolvable DID Document [recruiter] verifies VP created by [alice],passed,web-resolvable
8,2,reports/did-key/performance-report_2.json,3053,Evaluation - Phase 1 - Using did:key [recruiter] verifies VP with 1 derived VC created by [alice],passed,did-key,reports/web-resolvable/performance-report_2.json,98,Evaluation - Phase 1 - Using Web Resolvable DID Document [recruiter] verifies VP with 1 derived VC created by [alice],passed,web-resolvable
